In [1]:
using Distributed

using Pkg


@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [2]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 8

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 4000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100



 runnerdir = "/home/ghun245/RuNNer-testing/Brass_potential/"
 atomtype="Cu"
 pot = AbstractMLPotential(runnerdir,atomtype)
# RuNNer.getRuNNerenergy(runnerdir,1;input_idx=2)
pot_vector = []

for i =1:Threads.nthreads()
    potential=ParallelMLPotential(runnerdir,"Cu",i)
    push!(pot_vector,potential)
end

mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1] ; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
    

results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [4]:
function testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:4
        for j=1:100
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:4
        for j=1:100
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner (generic function with 1 method)

In [3]:
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]
#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu55 = copperconstant*ico_55
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886

# length(pos_cu55) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_cu55 = SphericalBC(radius=14*AtoBohr)   #5.32 Angstrom
bc_cu13 = SphericalBC(radius=8*AtoBohr)
#starting configuration
start_config = Config(pos_cu55, bc_cu55)
#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1]; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
mc_cycles = 4000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

MCParams(4000, 4000, 1, 8, 13, 100)

In [3]:
parallel_states,pot_vec,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

equilibration done


(Any[MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-0.4574246913329499, -0.1283289020784528, 0.34214466202378835], [-0.03364421914812554, 0.1421103508178847, 6.493762133196918], [1.6831246263298814, -5.618544933378812, 2.858963095689526], [-4.887936640409601, -3.527193132156385, 2.8340059896761303], [-4.943082569957563, 3.2149588765246757, 3.023935670109142], [2.017983310165621, 5.546995787072611, 2.9038996709289218], [5.770272457224476, -0.06547523001299947, 2.8588540740533337], [-1.6432101309141462, -5.733672010981994, -2.8147054322390983], [-5.540007266968018, 0.18036927020492644, -3.017190939731918], [-1.8464768784466157, 5.686226757805008, -2.8477515546317997], [4.70110409163636, 3.246747425064639, -3.1684917319461965], [4.781736515287305, -3.4805829333381832, -2.9164106923019175], [-0.09977371190151958, 0.32144971389790283, -6.3900998702

In [8]:
function testparallelrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:8
        for j=1:10
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:8
        for j=1:10
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner2 (generic function with 1 method)

In [11]:
@benchmark testrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 111.880 s (0.01% GC) to evaluate,
 with a memory estimate of 111.18 MiB, over 872044 allocations.

In [9]:
@benchmark testparallelrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 31.890 s (0.00% GC) to evaluate,
 with a memory estimate of 111.05 MiB, over 869430 allocations.

In [10]:
n_steps = 1

@benchmark ParallelRun.pptmc_cycle(parallel_states,mc_params,results,move_strat,pot_vec,ensemble,n_threads,delta_en,n_steps,a,v,r)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 157.047 s (0.11% GC) to evaluate,
 with a memory estimate of 555.95 MiB, over 4350115 allocations.

In [4]:
@time ParallelRun.pptmc_run!(mc_states,move_strat,mc_params,pot,ensemble,results)

equilibration done
1 cycles of 500 per 8 thread


cycle 1 of 1 complete
main loop complete

beginning statistics



done
3860.987124 seconds (47.54 M allocations: 5.790 GiB, 0.06% gc time, 0.06% compilation time)


In [5]:
parallel_states[1][1].max_displ


3-element Vector{Float64}:
 0.2012461179749811
 0.01
 1.0

In [25]:
results.en_avg

8-element Vector{Float64}:
 -48.969176570809914
 -48.7703578601413
 -48.64601842734141
 -48.0485124476216
 -48.383812474591075
 -48.42136870025184
 -48.38851476444973
 -48.330516106130624

In [24]:
println(results.en_histogram)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7835.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7801.0, 39.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,